### features:
* selected spectogram coef
* zero crossing rate
* selected chromogram values
* selected tempogram values
* wavelet of level 0

In [1]:
import pandas as pd 
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import auc
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, precision_recall_curve, recall_score
import matplotlib.pyplot as plt
import itertools

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
csv_path = "/Users/ecem/Desktop/csv of scgs"
wavelet_csv = "/Users/ecem/Desktop/wavelet-csv"

## Load data

Here, our target is to predict disease of patients with multiple diseases. So; our y will be disease column of two column indicatin geach patients disease.

In [3]:
spec = pd.read_csv(csv_path + '/p-value-selected-intersection-four-dis-spec.csv', index_col = 0)
#spec  #uncomment to see dataframe

signal_proc = pd.read_csv(csv_path + '/four-disease-final-signal_processing.csv', index_col = 0)

zero_crossing = signal_proc.iloc[:,:21]
#zero_crossing #uncomment to see dataframe

tempogram = pd.read_csv(csv_path + '/p-value-selected-four-dis-union-temp.csv', index_col = 0)
#tempogram #uncomment to see dataframe

chromogram = pd.read_csv(csv_path + '/p-value-selected-four-dis-union-chromogram.csv', index_col = 0)
#chromogram #uncomment to see dataframe

w_lvl0 = pd.read_csv(csv_path + '/four-disease-wavelet-lvl0-coef.csv', index_col = 0)
w_lvl0 = w_lvl0.reset_index(drop=True)
#w_lvl0  #uncomment to see dataframe

#wavelet =pd.read_csv(csv_path + '/p-value-selected-four-disease-mean-wavelet.csv', index_col = 0)

This line will be used to encode the target value. It is not going to be used for other purposes.

In [4]:
features = pd.concat([spec, w_lvl0], axis = 1)
features = pd.concat([features, zero_crossing], axis = 1)
features = pd.concat([features, chromogram], axis = 1)
features = pd.concat([features, tempogram], axis = 1)
#features = pd.concat([features, wavelet], axis = 1)

features

,freq_x1772,freq_y1773,freq_x1774,freq_y1774,freq_z554,freq_x555,freq_y555,freq_z555,freq_x556,freq_y556,...,tempogram_z106,tempogram_y112,tempogram_x116,tempogram_y207.1,tempogram_y227.1,tempogram_z246,tempogram_z253,tempogram_y353,tempogram_y370,tempogram_z374.1
0,0.001502,0.001610,0.001665,0.001679,0.001870,0.001893,0.001943,0.002011,0.002095,0.002203,...,6.188482e-17,4.392747e-17,6.259715e-17,-1.069911e-17,2.095696e-17,2.819607e-17,3.085184e-17,-3.298231e-17,-6.092841e-17,6.587582e-17
1,0.001071,0.001127,0.001137,0.001143,0.002001,0.001956,0.001950,0.001976,0.002043,0.002175,...,7.854615e-17,6.697528e-17,5.360961e-17,2.488668e-17,3.004509e-17,2.438228e-17,1.938164e-17,-1.824835e-17,-1.761129e-17,8.780803e-17
2,0.000985,0.001006,0.001021,0.001047,0.001530,0.001647,0.001771,0.001908,0.002053,0.002199,...,7.413183e-17,8.240297e-17,1.104261e-16,-2.567838e-17,1.563737e-17,8.424135e-18,2.059647e-17,-2.634202e-17,-3.694215e-17,8.083090e-17
3,0.001602,0.001491,0.001513,0.001518,0.001743,0.001808,0.001872,0.001936,0.002013,0.002083,...,4.562239e-17,4.845129e-17,4.446707e-17,-1.746719e-17,3.423315e-17,2.515351e-17,3.396500e-17,-3.469689e-17,-5.534130e-17,5.987236e-17
4,0.001523,0.001262,0.001153,0.001125,0.002226,0.002248,0.002256,0.002248,0.002224,0.002187,...,5.325780e-17,4.363806e-17,6.760344e-17,-3.850100e-18,3.678609e-17,5.717957e-17,4.738689e-17,-2.646103e-17,-7.969818e-17,7.040047e-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2387,0.000993,0.000940,0.000907,0.000890,0.000969,0.000953,0.000937,0.000914,0.000884,0.000848,...,5.347619e-17,7.485152e-17,8.153491e-17,-2.536445e-17,3.978511e-17,1.585965e-17,3.749178e-17,-3.705698e-17,-4.015822e-17,8.053797e-17
2388,0.000910,0.000878,0.000859,0.000849,0.000505,0.000532,0.000565,0.000599,0.000631,0.000658,...,5.171603e-17,6.871221e-17,4.617422e-17,-7.386951e-18,1.275095e-17,4.639175e-18,1.454632e-17,-1.868168e-17,-5.845148e-17,6.055644e-17
2389,0.000819,0.000744,0.000698,0.000681,0.000569,0.000544,0.000520,0.000502,0.000494,0.000496,...,5.518599e-17,9.277249e-17,6.856196e-17,1.090012e-17,2.964599e-17,5.220887e-17,4.991874e-17,-6.193681e-18,-2.759506e-17,7.286572e-17
2390,0.000812,0.000775,0.000785,0.000793,0.000720,0.000715,0.000708,0.000698,0.000686,0.000672,...,6.102847e-17,1.030882e-16,8.559952e-17,-1.899359e-17,1.097362e-17,1.122472e-17,6.336264e-18,2.768657e-18,-4.164722e-17,8.645078e-17


In [5]:
y = np.load(csv_path + "/four-disease-as-binary.npy")

In [6]:
y_df = pd.DataFrame(y)
y_df

,0,1,2,3
0,0,0,1,0
1,0,0,1,0
2,0,0,1,0
3,0,0,1,0
4,0,0,1,0
...,...,...,...,...
2387,0,1,0,0
2388,0,1,0,0
2389,0,1,0,0
2390,0,1,0,0


In [9]:
all_df = pd.concat([features, y_df], axis =1)
all_df

,freq_x1772,freq_y1773,freq_x1774,freq_y1774,freq_z554,freq_x555,freq_y555,freq_z555,freq_x556,freq_y556,...,tempogram_y227.1,tempogram_z246,tempogram_z253,tempogram_y353,tempogram_y370,tempogram_z374.1,0,1,2,3
0,0.001502,0.001610,0.001665,0.001679,0.001870,0.001893,0.001943,0.002011,0.002095,0.002203,...,2.095696e-17,2.819607e-17,3.085184e-17,-3.298231e-17,-6.092841e-17,6.587582e-17,0,0,1,0
1,0.001071,0.001127,0.001137,0.001143,0.002001,0.001956,0.001950,0.001976,0.002043,0.002175,...,3.004509e-17,2.438228e-17,1.938164e-17,-1.824835e-17,-1.761129e-17,8.780803e-17,0,0,1,0
2,0.000985,0.001006,0.001021,0.001047,0.001530,0.001647,0.001771,0.001908,0.002053,0.002199,...,1.563737e-17,8.424135e-18,2.059647e-17,-2.634202e-17,-3.694215e-17,8.083090e-17,0,0,1,0
3,0.001602,0.001491,0.001513,0.001518,0.001743,0.001808,0.001872,0.001936,0.002013,0.002083,...,3.423315e-17,2.515351e-17,3.396500e-17,-3.469689e-17,-5.534130e-17,5.987236e-17,0,0,1,0
4,0.001523,0.001262,0.001153,0.001125,0.002226,0.002248,0.002256,0.002248,0.002224,0.002187,...,3.678609e-17,5.717957e-17,4.738689e-17,-2.646103e-17,-7.969818e-17,7.040047e-17,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2387,0.000993,0.000940,0.000907,0.000890,0.000969,0.000953,0.000937,0.000914,0.000884,0.000848,...,3.978511e-17,1.585965e-17,3.749178e-17,-3.705698e-17,-4.015822e-17,8.053797e-17,0,1,0,0
2388,0.000910,0.000878,0.000859,0.000849,0.000505,0.000532,0.000565,0.000599,0.000631,0.000658,...,1.275095e-17,4.639175e-18,1.454632e-17,-1.868168e-17,-5.845148e-17,6.055644e-17,0,1,0,0
2389,0.000819,0.000744,0.000698,0.000681,0.000569,0.000544,0.000520,0.000502,0.000494,0.000496,...,2.964599e-17,5.220887e-17,4.991874e-17,-6.193681e-18,-2.759506e-17,7.286572e-17,0,1,0,0
2390,0.000812,0.000775,0.000785,0.000793,0.000720,0.000715,0.000708,0.000698,0.000686,0.000672,...,1.097362e-17,1.122472e-17,6.336264e-18,2.768657e-18,-4.164722e-17,8.645078e-17,0,1,0,0


In [10]:
small = all_df.sample(n=1150).reset_index().drop(['index'], axis =1)
small

,freq_x1772,freq_y1773,freq_x1774,freq_y1774,freq_z554,freq_x555,freq_y555,freq_z555,freq_x556,freq_y556,...,tempogram_y227.1,tempogram_z246,tempogram_z253,tempogram_y353,tempogram_y370,tempogram_z374.1,0,1,2,3
0,0.000914,0.001015,0.001089,0.001137,0.001668,0.001602,0.001545,0.001505,0.001488,0.001490,...,8.341099e-18,1.569370e-17,4.268746e-17,-3.182581e-17,-6.515980e-17,6.633810e-17,1,0,1,0
1,0.001429,0.001547,0.001537,0.001515,0.001260,0.001290,0.001317,0.001351,0.001385,0.001413,...,7.858806e-17,6.387237e-17,1.137845e-16,-3.507649e-17,-4.676978e-17,8.669471e-17,1,0,0,0
2,0.001109,0.001249,0.001313,0.001333,0.001767,0.001833,0.001892,0.001935,0.001958,0.001959,...,4.660380e-17,2.822232e-17,3.875336e-17,-4.830096e-17,-7.852620e-17,6.419101e-17,1,0,1,0
3,0.001532,0.001701,0.001804,0.001855,0.002991,0.002955,0.002922,0.002947,0.003009,0.003102,...,1.991407e-17,1.305639e-17,2.169482e-17,-2.931980e-17,-5.881504e-17,5.467281e-17,0,0,0,1
4,0.003883,0.003459,0.003318,0.003290,0.001892,0.001925,0.001969,0.002041,0.002131,0.002226,...,-3.091561e-19,2.338233e-18,2.294922e-17,-3.421747e-19,-3.600530e-17,1.919480e-17,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1145,0.003483,0.003907,0.003839,0.003746,0.007060,0.007010,0.006867,0.006638,0.006333,0.005968,...,1.258920e-17,-2.120431e-18,2.340268e-17,-2.488462e-17,-7.731605e-17,6.238419e-17,0,0,0,1
1146,0.001045,0.001141,0.001256,0.001331,0.002906,0.002981,0.003031,0.003053,0.003056,0.003031,...,1.326228e-17,3.094238e-17,4.378891e-17,-2.943958e-17,-5.825928e-17,5.788835e-17,0,0,1,0
1147,0.001211,0.001261,0.001344,0.001390,0.002092,0.002141,0.002167,0.002172,0.002168,0.002151,...,4.806430e-17,5.109068e-17,-1.771778e-17,-4.644828e-18,-4.933862e-17,8.308966e-17,0,1,1,1
1148,0.001326,0.001048,0.000916,0.000863,0.001843,0.001844,0.001845,0.001838,0.001823,0.001826,...,5.422647e-17,4.654582e-17,3.176477e-17,-1.602675e-18,-3.985877e-17,8.340125e-17,1,0,1,0


In [15]:
y_random = small.iloc[:,-4:]
y_random

,0,1,2,3
0,1,0,1,0
1,1,0,0,0
2,1,0,1,0
3,0,0,0,1
4,0,1,1,0
...,...,...,...,...
1145,0,0,0,1
1146,0,0,1,0
1147,0,1,1,1
1148,1,0,1,0


In [21]:
random_df = small.iloc[:,:763]
random_df

,freq_x1772,freq_y1773,freq_x1774,freq_y1774,freq_z554,freq_x555,freq_y555,freq_z555,freq_x556,freq_y556,...,tempogram_z106,tempogram_y112,tempogram_x116,tempogram_y207.1,tempogram_y227.1,tempogram_z246,tempogram_z253,tempogram_y353,tempogram_y370,tempogram_z374.1
0,0.000914,0.001015,0.001089,0.001137,0.001668,0.001602,0.001545,0.001505,0.001488,0.001490,...,4.175964e-17,4.635517e-17,3.801757e-17,-2.797615e-17,8.341099e-18,1.569370e-17,4.268746e-17,-3.182581e-17,-6.515980e-17,6.633810e-17
1,0.001429,0.001547,0.001537,0.001515,0.001260,0.001290,0.001317,0.001351,0.001385,0.001413,...,3.016895e-17,8.991809e-17,6.358261e-17,-3.831237e-17,7.858806e-17,6.387237e-17,1.137845e-16,-3.507649e-17,-4.676978e-17,8.669471e-17
2,0.001109,0.001249,0.001313,0.001333,0.001767,0.001833,0.001892,0.001935,0.001958,0.001959,...,9.050870e-17,7.885407e-17,7.417560e-17,-6.903228e-17,4.660380e-17,2.822232e-17,3.875336e-17,-4.830096e-17,-7.852620e-17,6.419101e-17
3,0.001532,0.001701,0.001804,0.001855,0.002991,0.002955,0.002922,0.002947,0.003009,0.003102,...,6.074100e-17,1.022592e-16,7.796886e-17,1.791470e-17,1.991407e-17,1.305639e-17,2.169482e-17,-2.931980e-17,-5.881504e-17,5.467281e-17
4,0.003883,0.003459,0.003318,0.003290,0.001892,0.001925,0.001969,0.002041,0.002131,0.002226,...,5.998124e-17,1.028730e-16,1.035983e-16,-2.265715e-17,-3.091561e-19,2.338233e-18,2.294922e-17,-3.421747e-19,-3.600530e-17,1.919480e-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1145,0.003483,0.003907,0.003839,0.003746,0.007060,0.007010,0.006867,0.006638,0.006333,0.005968,...,6.037393e-17,9.818756e-17,7.752160e-17,-3.986219e-17,1.258920e-17,-2.120431e-18,2.340268e-17,-2.488462e-17,-7.731605e-17,6.238419e-17
1146,0.001045,0.001141,0.001256,0.001331,0.002906,0.002981,0.003031,0.003053,0.003056,0.003031,...,5.825378e-17,6.241840e-17,7.484834e-17,-3.348386e-17,1.326228e-17,3.094238e-17,4.378891e-17,-2.943958e-17,-5.825928e-17,5.788835e-17
1147,0.001211,0.001261,0.001344,0.001390,0.002092,0.002141,0.002167,0.002172,0.002168,0.002151,...,3.243062e-17,1.046195e-16,5.631915e-17,-9.027178e-18,4.806430e-17,5.109068e-17,-1.771778e-17,-4.644828e-18,-4.933862e-17,8.308966e-17
1148,0.001326,0.001048,0.000916,0.000863,0.001843,0.001844,0.001845,0.001838,0.001823,0.001826,...,5.003209e-17,3.564191e-17,5.143064e-17,-1.018279e-17,5.422647e-17,4.654582e-17,3.176477e-17,-1.602675e-18,-3.985877e-17,8.340125e-17


## With threshold

In [22]:
from sklearn.feature_selection import VarianceThreshold
vt = VarianceThreshold(threshold =  0.0001)   # düşük  threshold
_ = vt.fit(random_df)
masked = vt.get_support()
features_masked = random_df.loc[:, masked]
features_masked # uncomment to see the dataframe

,freq_x1939,freq_y1939,freq_z563,freq_x564,freq_y564,freq_z564,freq_x565,freq_y565,freq_z565,freq_x566,...,A_x.1,A_y,A_z,A♯_y,A♯_z,B_x,B_y,B_z,tempogram_y256,tempogram_z256
0,0.003845,0.003737,0.001249,0.001226,0.001202,0.001173,0.001142,0.001116,0.001105,0.001107,...,0.601098,0.661589,0.741489,0.943612,0.985775,0.961503,0.970383,0.931369,0.629736,0.628924
1,0.001319,0.001329,0.001496,0.001498,0.001492,0.001481,0.001480,0.001478,0.001469,0.001451,...,0.678827,0.729680,0.750834,0.849146,0.962369,0.937439,0.920995,1.000000,0.969079,0.957505
2,0.003871,0.003718,0.001371,0.001343,0.001324,0.001321,0.001346,0.001386,0.001436,0.001489,...,0.683303,0.711207,0.734991,0.849411,0.963371,0.966144,0.968898,0.956779,0.963524,0.953013
3,0.002235,0.002187,0.002914,0.002879,0.002850,0.002820,0.002792,0.002790,0.002809,0.002844,...,0.718587,0.615381,0.655372,0.666221,0.779939,0.931638,0.968285,0.951536,0.487206,0.476227
4,0.004994,0.005274,0.001913,0.001917,0.001913,0.001902,0.001891,0.001887,0.001882,0.001887,...,0.602881,0.584158,0.495895,0.555750,0.701913,0.973836,0.828903,0.651053,0.450369,0.222096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1145,0.006636,0.006098,0.007379,0.007208,0.006960,0.006673,0.006326,0.005918,0.005464,0.004989,...,0.840361,0.959511,0.945556,0.887339,0.823650,0.995016,0.996707,0.814848,0.554720,0.555865
1146,0.002565,0.002695,0.002406,0.002330,0.002267,0.002207,0.002156,0.002117,0.002089,0.002061,...,0.503644,0.560860,0.699685,0.836645,0.963102,0.872068,0.951128,0.960488,0.560848,0.554925
1147,0.005582,0.005225,0.002267,0.002361,0.002436,0.002491,0.002529,0.002548,0.002540,0.002503,...,0.846396,0.814465,0.782048,0.765028,0.924970,0.892945,1.000000,0.918989,0.543172,0.525885
1148,0.004131,0.003961,0.001553,0.001512,0.001494,0.001493,0.001512,0.001549,0.001605,0.001689,...,0.484646,0.612507,0.694885,0.900063,0.961267,0.986885,0.903160,0.893107,0.616833,0.606551


In [23]:
X_train_, X_valid_, y_train_, y_valid_ = train_test_split(features_masked, y_random.to_numpy(), train_size=0.8, test_size=0.2,
                                                                random_state=42)

In [24]:
import xgboost as xgb
classifier = MultiOutputClassifier(xgb.XGBClassifier(objective = 'binary:logistic'))

clf = Pipeline([('classify', classifier)])


In [25]:
clf.fit(X_train_, y_train_)


Pipeline(steps=[('classify',
                 MultiOutputClassifier(estimator=XGBClassifier(base_score=None,
                                                               booster=None,
                                                               callbacks=None,
                                                               colsample_bylevel=None,
                                                               colsample_bynode=None,
                                                               colsample_bytree=None,
                                                               early_stopping_rounds=None,
                                                               enable_categorical=False,
                                                               eval_metric=None,
                                                               gamma=None,
                                                               gpu_id=None,
                                                               grow_policy=None,
                                                               importance_type=None,
                                                               interaction_constraints=None,
                                                               learning_rate=None,
                                                               max_bin=None,
                                                               max_cat_to_onehot=None,
                                                               max_delta_step=None,
                                                               max_depth=None,
                                                               max_leaves=None,
                                                               min_child_weight=None,
                                                               missing=nan,
                                                               monotone_constraints=None,
                                                               n_estimators=100,
                                                               n_jobs=None,
                                                               num_parallel_tree=None,
                                                               predictor=None,
                                                               random_state=None,
                                                               reg_alpha=None,
                                                               reg_lambda=None, ...)))])

In [26]:
yhat = clf.predict(X_valid_)

In [27]:
df_results_test = pd.DataFrame({'AR': y_valid_[:,0], 'AS': y_valid_[:,1], 'MR': y_valid_[:,2], 'MS': y_valid_[:,3],
                        'predicted AR': yhat[:,0],
                          'predicted AS': yhat[:,1],
                          'predicted MR': yhat[:,2], 
                          'predicted MS': yhat[:,3]})
df_results_test

,AR,AS,MR,MS,predicted AR,predicted AS,predicted MR,predicted MS
0,0,0,0,1,0,0,0,1
1,1,0,1,0,1,0,1,0
2,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,1,0
4,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...
225,0,1,0,0,0,1,0,0
226,0,1,0,0,0,1,0,0
227,0,0,0,1,0,0,0,1
228,0,0,1,0,0,0,1,0


In [28]:
accuracy = clf.score(X_valid_, y_valid_)
accuracy

0.808695652173913

In [29]:
y_valid_[:,0]

array([0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [30]:
yhat[:,0]

array([0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0,
       1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0])

In [31]:
auc_y1 = roc_auc_score(y_valid_[:,0],yhat[:,0])
auc_y2 = roc_auc_score(y_valid_[:,1],yhat[:,1])
auc_y3 = roc_auc_score(y_valid_[:,2],yhat[:,2])
auc_y4 = roc_auc_score(y_valid_[:,3],yhat[:,3])

print("ROC AUC AR: %.4f, AS: %.4f, MR: %.4f, MS: %.4f " % (auc_y1, auc_y2, auc_y3, auc_y4))


ROC AUC AR: 0.8987, AS: 0.9358, MR: 0.9045, MS: 0.8722 


In [48]:
cm_y1 = confusion_matrix(y_valid_[:,0],yhat[:,0])
cm_y2 = confusion_matrix(y_valid_[:,1],yhat[:,1])
cm_y3 = confusion_matrix(y_valid_[:,2],yhat[:,2])
cm_y4 = confusion_matrix(y_valid_[:,3],yhat[:,3])
print(cm_y1)



[[321   3]
 [ 20 135]]


In [49]:
cm_y2

array([[267,   8],
       [ 12, 192]])

In [50]:
cm_y3

array([[226,  12],
       [ 14, 227]])

In [51]:
cm_y4

array([[383,   3],
       [  8,  85]])

In [52]:
cr_y1 = classification_report(y_valid_[:,0],yhat[:,0]) # AR
cr_y2 = classification_report(y_valid_[:,1],yhat[:,1]) # AS
cr_y3 = classification_report(y_valid_[:,2],yhat[:,2]) # MR
cr_y4 = classification_report(y_valid_[:,3],yhat[:,3]) # MS

print (cr_y1)
print (cr_y2)
print (cr_y3)
print (cr_y4)




              precision    recall  f1-score   support

           0       0.94      0.99      0.97       324
           1       0.98      0.87      0.92       155

    accuracy                           0.95       479
   macro avg       0.96      0.93      0.94       479
weighted avg       0.95      0.95      0.95       479

              precision    recall  f1-score   support

           0       0.96      0.97      0.96       275
           1       0.96      0.94      0.95       204

    accuracy                           0.96       479
   macro avg       0.96      0.96      0.96       479
weighted avg       0.96      0.96      0.96       479

              precision    recall  f1-score   support

           0       0.94      0.95      0.95       238
           1       0.95      0.94      0.95       241

    accuracy                           0.95       479
   macro avg       0.95      0.95      0.95       479
weighted avg       0.95      0.95      0.95       479

              preci

## Without threshold

In [18]:
X_train, X_valid, y_train, y_valid = train_test_split(features, y, train_size=0.8, test_size=0.2,
                                                                random_state=42)

In [19]:
classifier_f = MultiOutputClassifier(xgb.XGBClassifier(objective = 'binary:logistic'))

clf_f = Pipeline([('classify', classifier)])

clf_f.fit(X_train, y_train)

yhat_f = clf.predict(X_valid)

In [20]:
df_results_test_f = pd.DataFrame({'AR': y_valid[:,0], 'AS': y_valid[:,1], 'MR': y_valid[:,2], 'MS': y_valid[:,3],
                        'predicted AR': yhat_f[:,0],
                          'predicted AS': yhat_f[:,1],
                          'predicted MR': yhat_f[:,2], 
                          'predicted MS': yhat_f[:,3]})
df_results_test_f

,AR,AS,MR,MS,predicted AR,predicted AS,predicted MR,predicted MS
0,0,0,1,0,0,0,1,0
1,1,1,0,0,0,0,0,0
2,0,1,0,0,0,1,0,0
3,0,0,1,0,0,0,1,0
4,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...
474,1,0,1,0,1,0,1,0
475,0,0,1,0,0,0,1,0
476,0,0,0,0,0,0,0,0
477,1,0,0,0,1,0,0,0


In [21]:
auc_y1_ = roc_auc_score(y_valid[:,0],yhat_f[:,0])
auc_y2_ = roc_auc_score(y_valid[:,1],yhat_f[:,1])
auc_y3_ = roc_auc_score(y_valid[:,2],yhat_f[:,2])
auc_y4_ = roc_auc_score(y_valid[:,3],yhat_f[:,3])

print("ROC AUC AR: %.4f, AS: %.4f, MR: %.4f, MS: %.4f " % (auc_y1_, auc_y2_, auc_y3_, auc_y4_))

ROC AUC AR: 0.9454, AS: 0.9579, MR: 0.9604, MS: 0.9221 


In [22]:
cr_y1_ = classification_report(y_valid[:,0],yhat_f[:,0]) # AR
cr_y2_ = classification_report(y_valid[:,1],yhat_f[:,1]) # AS
cr_y3_ = classification_report(y_valid[:,2],yhat_f[:,2]) #mr
cr_y4_ = classification_report(y_valid[:,3],yhat_f[:,3]) #MS

print (cr_y1_)
print (cr_y2_)
print (cr_y3_)
print (cr_y4_)



              precision    recall  f1-score   support

           0       0.96      0.99      0.97       324
           1       0.97      0.90      0.94       155

    accuracy                           0.96       479
   macro avg       0.96      0.95      0.95       479
weighted avg       0.96      0.96      0.96       479

              precision    recall  f1-score   support

           0       0.96      0.97      0.97       275
           1       0.96      0.94      0.95       204

    accuracy                           0.96       479
   macro avg       0.96      0.96      0.96       479
weighted avg       0.96      0.96      0.96       479

              precision    recall  f1-score   support

           0       0.95      0.97      0.96       238
           1       0.97      0.95      0.96       241

    accuracy                           0.96       479
   macro avg       0.96      0.96      0.96       479
weighted avg       0.96      0.96      0.96       479

              preci